In [2]:
import librosa
import librosa.display
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import load_dataset
from scipy.io.wavfile import read
import io
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import tensorflow as tf

# Check device for PyTorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device (PyTorch):", device)

# Check device for TensorFlow
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print("GPU is available (TensorFlow)")
else:
    print("GPU is not available (TensorFlow)")

Using device (PyTorch): cuda
GPU is available (TensorFlow)


In [4]:
from datasets import config

# Increase the timeout to 30 minutes (1800 seconds)
config.DOWNLOAD_TIMEOUT = 1800

# Load the dataset
from datasets import load_dataset
dataset = load_dataset("AbstractTTS/IEMOCAP")


In [ ]:
print(dataset)
print(dataset['train'][0])  # Display the first


In [ ]:
# Convert the 'train' split to a DataFrame
df = pd.DataFrame(dataset['train'])

# Inspect the DataFrame
print(df.head())  # Check the first few rows of the DataFrame
print(df.columns)  # Verify the column names


In [ ]:
# Convert dataset['train'] to a Pandas DataFrame
df = dataset['train'].to_pandas()
# Inspect the DataFrame
print(df.head())
print(df.columns)


In [ ]:
print(df['major_emotion'].unique())  # Check unique emotion labels
print(df['major_emotion'].isnull().sum())  # Check for missing values

In [ ]:
def plot_emotion_distribution(df):
    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 6))
    plt.title('Count of Emotions', size=16)
    sns.countplot(data=df, x='major_emotion', palette='viridis')  # Replace 'major_emotion' with the actual column name
    plt.ylabel('Count', size=12)
    plt.xlabel('Emotions', size=12)
    sns.despine(top=True, right=True)
    plt.show()

# Call the function
plot_emotion_distribution(df)


In [18]:
def plot_emotion_distribution(df):
    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 6))
    plt.title('Count of Emotions', size=16)
    # Sort emotions by count
    order = df['major_emotion'].value_counts().index
    sns.countplot(data=df, x='major_emotion', order=order, palette='viridis')
    plt.ylabel('Count', size=12)
    plt.xlabel('Emotions', size=12)
    sns.despine(top=True, right=True)
    plt.show()


In [ ]:
plot_emotion_distribution(df)


In [ ]:
print(df['file'].head())


In [ ]:
def extract_mfcc(audio_array, sampling_rate, n_mfcc=13):
    mfccs = librosa.feature.mfcc(y=audio_array, sr=sampling_rate, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs, axis=1)  # Take the mean across time
    return mfccs_mean

# Prepare a list to store MFCCs
mfcc_list = []

# Iterate over the audio files in the dataset
for idx, row in df.iterrows():
    audio_path = row['audio']['path']  # Get the file path
    audio_array = np.array(row['audio']['array'])  # Get the audio array
    sampling_rate = row['audio']['sampling_rate']  # Get the sampling rate

    # Extract MFCC features
    mfcc_features = extract_mfcc(audio_array, sampling_rate)

    # Append to the list
    mfcc_list.append(mfcc_features)

# Add MFCC features to the DataFrame
df['mfcc'] = mfcc_list

# Save the DataFrame with MFCCs to a CSV file
df.to_csv("iemocap_with_mfcc.csv", index=False)

print("MFCC extraction complete. Data saved to 'iemocap_with_mfcc.csv'")


In [ ]:
print(df['mfcc'].head())  # Verify MFCC values
print(df['mfcc'].isnull().sum())  # Check for missing values


In [ ]:
print(df['major_emotion'].unique())  # Check unique emotion labels
print(df['major_emotion'].isnull().sum())  # Check for missing labels


In [ ]:
print(df['major_emotion'].unique())  # Check unique emotion labels
print(df['major_emotion'].isnull().sum())  # Check for missing labels


In [ ]:
# Load the dataset
Features = pd.read_csv('../audio/iemocap_with_mfcc.csv')

# Verify the structure of the dataset
print(Features.head())  # Check the first few rows
print(Features.info())  # Check for null values and column information
print(Features['major_emotion'].unique())  # Verify unique label values in the correct column

# Use 'major_emotion' as the label column
X = Features['mfcc'].apply(lambda x: np.fromstring(x.strip('[]'), sep=' '))  # Convert MFCC strings to numpy arrays
Y = Features['major_emotion']  # Use the 'major_emotion' column as labels

# Verify the shapes of the features and labels
print(f"Number of samples: {len(X)}")
print(f"Number of labels: {len(Y)}")


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encode labels
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

# Convert features to numpy array and pad/truncate to fixed size
X_array = np.array([np.pad(mfcc, (0, 13 - len(mfcc)), 'constant') if len(mfcc) < 13 else mfcc[:13] for mfcc in X])

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_array, Y_encoded, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")


In [ ]:
print(f"Number of training samples: {len(X_train)}")
print(f"Number of testing samples: {len(X_test)}")
print(f"Unique classes: {len(np.unique(y_train))}")
print(f"Sample feature (X_train[0]): {X_train[0]}")
print(f"Sample label (y_train[0]): {y_train[0]}")


In [145]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Ensure MFCC column contains arrays
if isinstance(df['mfcc'].iloc[0], str):  # Check if the first element is a string
    X = df['mfcc'].apply(lambda x: np.fromstring(x.strip('[]'), sep=' '))  # Convert strings to numpy arrays
else:
    X = df['mfcc']  # Directly use the numpy arrays if already in array format

# Labels
Y = df['major_emotion']

# Encode labels
encoder = OneHotEncoder()
Y_encoded = encoder.fit_transform(np.array(Y).reshape(-1, 1)).toarray()

# Pad/truncate MFCC features
X_array = np.array([np.pad(mfcc, (0, 13 - len(mfcc)), 'constant') if len(mfcc) < 13 else mfcc[:13] for mfcc in X])

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_array)

# Expand dimensions for CNN
X_scaled = np.expand_dims(X_scaled, axis=2)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y_encoded, test_size=0.2, random_state=42)


In [ ]:
# Build the model
model = Sequential()
model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))

model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))
model.add(Dropout(0.2))

model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(units=y_train.shape[1], activation='softmax'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Set up learning rate reduction
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, patience=2, min_lr=1e-7)

# Train the model
history = model.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_test, y_test), callbacks=[rlrp])


In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

# Print the test accuracy
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")     

In [ ]:
print(df['audio'].iloc[0])

In [168]:
local_model_dir = r"C:\Users\dell\Desktop\gradproject\AI04-Grad\audio\wav2vec2-large-xlsr-53"

In [169]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model

# Load the processor and model from the local directory
processor = Wav2Vec2Processor.from_pretrained(local_model_dir)
wav2vec_model = Wav2Vec2Model.from_pretrained(local_model_dir)


## way one 

In [170]:
import numpy as np
import torch

# Example function to process audio data
def process_audio(row):
    # Extract audio array and sampling rate
    audio_array = row['audio']['array']
    sampling_rate = row['audio']['sampling_rate']
    
    # Preprocess the audio
    inputs = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    
    # Pass through the Wav2Vec2 model
    with torch.no_grad():
        outputs = wav2vec_model(**inputs)
    
    # Take the mean of the last hidden states (average pooling)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings


In [ ]:
# Apply the feature extraction to all rows in the DataFrame
df['embeddings'] = df.apply(process_audio, axis=1)

# Save the embeddings and labels for training
X = np.stack(df['embeddings'].values)
Y = df['major_emotion'].values


## way two

In [32]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model

# Path to the locally saved model files
local_model_dir = r"C:\Users\dell\Desktop\gradproject\AI04-Grad\audio\wav2vec2-large-xlsr-53"

# Load the feature extractor and model
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(local_model_dir)
wav2vec_model = Wav2Vec2Model.from_pretrained(local_model_dir)


In [ ]:
print(df.head())
print(df['audio'].iloc[0])  # Inspect the first row's 'audio' field


In [ ]:
import torch
from tqdm import tqdm
from scipy.io import wavfile
import io
import numpy as np

# Prepare lists to store embeddings and labels
X_features = []
y_labels = []

# Iterate through the dataset
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    try:
        # Extract the raw bytes from the audio field
        audio_bytes = row['audio']['bytes']
        label = row['major_emotion']  # Extract the emotion label

        # Decode the bytes into a waveform
        sampling_rate, audio_array = wavfile.read(io.BytesIO(audio_bytes))
        audio_array = audio_array.astype(np.float32)  # Convert to float32 for compatibility

        # Extract features using the Wav2Vec2 feature extractor
        inputs = feature_extractor(audio_array, return_tensors="pt", sampling_rate=sampling_rate)
        with torch.no_grad():
            outputs = wav2vec_model(**inputs)

        # Average pooling to get a single vector for the audio
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

        # Append features and label to lists
        X_features.append(embeddings)
        y_labels.append(label)

    except Exception as e:
        print(f"[ERROR] Failed to process index {idx}: {e}")

print("Feature extraction complete.")


In [ ]:
print(df['audio'].head())  # Inspect the audio column
print(df['audio'].iloc[0])  # Check the first entry in detail


In [ ]:
import librosa
import soundfile as sf
import os
import numpy as np
from datasets import load_dataset

# Load the IEMOCAP dataset
dataset = load_dataset("AbstractTTS/IEMOCAP")
df = dataset['train'].to_pandas()  # Convert to Pandas DataFrame

# Output directory for preprocessed audio
output_dir = "./preprocessed_audio/"
os.makedirs(output_dir, exist_ok=True)

def preprocess_and_save(audio_bytes, output_path, target_sampling_rate=16000):
    """Decode, resample, and save audio."""
    # Decode raw audio bytes to a NumPy array
    audio_array = np.frombuffer(audio_bytes, dtype=np.int16).astype(np.float32) / 32768.0  # Normalize to [-1, 1]
    
    # Resample the audio
    audio_resampled = librosa.resample(audio_array, orig_sr=16000, target_sr=target_sampling_rate)
    
    # Save the resampled audio
    sf.write(output_path, audio_resampled, target_sampling_rate)

# Process each audio sample in the dataset
for idx, row in df.iterrows():
    audio_bytes = row['audio']['bytes']  # Extract the raw audio bytes
    file_name = row['file']  # Use the original filename
    
    # Define output file path
    output_path = os.path.join(output_dir, file_name)
    
    # Preprocess and save audio
    preprocess_and_save(audio_bytes, output_path)

print("Preprocessing complete. Audio files saved in:", output_dir)


In [ ]:
import os

# Verify preprocessed files
print(f"Number of preprocessed files: {len(os.listdir(output_dir))}")


In [ ]:
import librosa
import numpy as np
import pandas as pd
import os

# Output file for features
featuresIEMOCAP_csv = "audio_features.csv"

# Prepare lists to store features and labels
features = []
labels = []

print("Starting feature extraction...")

# Iterate through preprocessed audio files
for idx, row in df.iterrows():
    file_name = row['file']
    label = row['major_emotion']
    file_path = os.path.join(output_dir, file_name)
    
    # Check if file exists
    if not os.path.exists(file_path):
        print(f"[WARNING] File not found: {file_path}. Skipping...")
        continue
    
    try:
        # Load preprocessed audio
        audio, sr = librosa.load(file_path, sr=16000)
        
        # Extract MFCC features
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)  # Take mean across time
        
        # Append features and labels
        features.append(mfcc_mean)
        labels.append(label)
        
        # Log progress every 100 files
        if idx % 100 == 0:
            print(f"[INFO] Processed {idx}/{len(df)} files...")
    except Exception as e:
        print(f"[ERROR] Failed to process file: {file_path}. Error: {str(e)}")

# Convert features and labels to DataFrame
features_df = pd.DataFrame(features)
features_df['label'] = labels

# Save to CSV   
features_df.to_csv(featuresIEMOCAP_csv, index=False)

print(f"Feature extraction complete. Features saved to {featuresIEMOCAP_csv}")


In [17]:
import pandas as pd

# Load the extracted features
features_df = pd.read_csv("audio_features.csv")

# Separate features and labels
X = features_df.iloc[:, :-1].values  # Extract features (all columns except the last)
y = features_df['label'].values  # Extract labels (last column)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Save the label mapping for later use
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print("Label mapping:", label_mapping)


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")


In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Test the model
y_pred = clf.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))


In [23]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)  # Adjust the learning rate
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from tensorflow.keras.layers import BatchNormalization, LeakyReLU

model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation=LeakyReLU(alpha=0.1)),
    Dropout(0.3),
    Dense(len(label_mapping), activation='softmax')
])


# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define the model
model = Sequential([
    Dense(256, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.4),  # Increased dropout for regularization
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation=LeakyReLU(alpha=0.1), kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(len(label_mapping), activation='softmax')  # Output layer
])

# Compile the model with tuned hyperparameters
optimizer = Adam(learning_rate=0.001)  # Start with a lower learning rate
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),  # Stop early if no improvement
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)  # Reduce learning rate on plateau
]

# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=50,  # Allow more epochs for better convergence
                    batch_size=32,
                    callbacks=callbacks)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

# Visualize training progress
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [25]:
# Reshape your data for GRU
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # Add a single feature dimension if needed
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define the GRU-based model
model = Sequential([
    GRU(256, return_sequences=True, kernel_regularizer=l2(0.001), input_shape=(X_train.shape[1], X_train.shape[2])),
    BatchNormalization(),
    Dropout(0.4),  # Increased dropout for regularization
    
    GRU(128, return_sequences=False, kernel_regularizer=l2(0.001)),  # Final GRU layer without returning sequences
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    
    Dense(len(label_mapping), activation='softmax')  # Output layer
])

# Compile the model with tuned hyperparameters
optimizer = Adam(learning_rate=0.001)  # Start with a lower learning rate
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),  # Stop early if no improvement
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)  # Reduce learning rate on plateau
]

# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=50,  # Allow more epochs for better convergence
                    batch_size=32,
                    callbacks=callbacks)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

# Visualize training progress
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# Load features and labels
Features = pd.read_csv('features.csv')

# Prepare data
X = Features.iloc[:, :-1].values  # Features
Y = Features['label'].values  # Labels

# One-hot encode labels
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1, 1)).toarray()

# Split the data
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, shuffle=True)

# Scale the data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Reshape data for CNN
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

# Build the CNN model
model = Sequential([
    Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(x_train.shape[1], 1)),
    BatchNormalization(),
    MaxPooling1D(pool_size=5, strides=2, padding='same'),
    Dropout(0.3),
    
    Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=5, strides=2, padding='same'),
    Dropout(0.3),
    
    Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=5, strides=2, padding='same'),
    Dropout(0.3),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(len(encoder.categories_[0]), activation='softmax')  # Output layer
])

# Compile the model
optimizer = Adam(learning_rate=0.001)  # Adjusted learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
]

# Train the model
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=64,
    epochs=50,
    callbacks=callbacks
)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", accuracy)

# Plot training and validation metrics
epochs = range(len(history.history['accuracy']))
plt.figure(figsize=(20, 6))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['accuracy'], label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['loss'], label='Training Loss')
plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Predict on test data
y_pred = np.argmax(model.predict(x_test), axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Create a DataFrame for comparison
df_results = pd.DataFrame({'Predicted Labels': encoder.inverse_transform(y_pred.reshape(-1, 1)).flatten(),
                           'Actual Labels': encoder.inverse_transform(y_test.reshape(-1, len(y_test[0]))).flatten()})

# Confusion matrix
cm = confusion_matrix(y_test_labels, y_pred)
cm_df = pd.DataFrame(cm, index=encoder.categories_[0], columns=encoder.categories_[0])

# Plot confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', linewidths=0.5)
plt.title('Confusion Matrix', size=20)
plt.xlabel('Predicted Labels', size=14)
plt.ylabel('Actual Labels', size=14)
plt.show()

# Print classification report
print(classification_report(y_test_labels, y_pred, target_names=encoder.categories_[0]))


In [ ]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model
import torch
import numpy as np
from tqdm import tqdm  # for progress tracking

# Path to your local directory containing the model files
local_model_dir = r"C:\Users\dell\Desktop\gradproject\AI04-Grad\audio\wav2vec2-large-xlsr-53"

# Initialize Wav2Vec2 feature extractor and model from the local directory
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(local_model_dir)
wav2vec_model = Wav2Vec2Model.from_pretrained(local_model_dir)

X_train_wav2vec = []
X_test_wav2vec = []

# Assuming `df` is your dataset
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    audio_array = row['audio']['array']  # Extract the audio array
    sampling_rate = row['audio']['sampling_rate']  # Extract the sampling rate

    # Extract features using Wav2Vec2 feature extractor
    inputs = feature_extractor(audio_array, return_tensors="pt", sampling_rate=sampling_rate)
    with torch.no_grad():
        outputs = wav2vec_model(**inputs)
    
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()  # Average pooling

    if idx < len(df) * 0.8:  # Assuming 80% for training
        X_train_wav2vec.append(embeddings)
    else:
        X_test_wav2vec.append(embeddings)

# Stack the features into tensors
X_train_wav2vec = torch.stack(X_train_wav2vec)
X_test_wav2vec = torch.stack(X_test_wav2vec)

print("Feature extraction complete.")


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

# Example model
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train_wav2vec.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(df['major_emotion'].nunique(), activation='softmax'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Learning rate reduction
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, patience=2, min_lr=1e-7)

# Train the model
history = model.fit(X_train_wav2vec, df['major_emotion'][:len(X_train_wav2vec)],
                    batch_size=64, epochs=50, validation_data=(X_test_wav2vec, df['major_emotion'][len(X_train_wav2vec):]),
                    callbacks=[rlrp])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_wav2vec, df['major_emotion'][len(X_train_wav2vec):])
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


In [ ]:
from vggish import vggish_input, vggish_params, vggish_postprocess
import numpy as np

# Load VGGish model parameters
vggish_params = vggish_params.VggishParams()
vggish_model_path = "vggish_model.ckpt"  # Path to pre-trained VGGish model

# Extract VGGish features
X_train_vggish = []
X_test_vggish = []

for idx, row in df.iterrows():
    audio_array = row['audio']['array']
    input_features = vggish_input.wavfile_to_examples(audio_array, vggish_params)
    vggish_scores = vggish_postprocess.run(input_features)
    if idx < len(X_train):  # Split into train and test
        X_train_vggish.append(vggish_scores)
    else:
        X_test_vggish.append(vggish_scores)

X_train_vggish = np.vstack(X_train_vggish)
X_test_vggish = np.vstack(X_test_vggish)


In [ ]:
# Define the model with extracted features
model_wav2vec = Sequential()
model_wav2vec.add(Dense(256, activation='relu', input_shape=(X_train_wav2vec.shape[1],)))
model_wav2vec.add(Dropout(0.3))
model_wav2vec.add(Dense(128, activation='relu'))
model_wav2vec.add(Dropout(0.3))
model_wav2vec.add(Dense(64, activation='relu'))
model_wav2vec.add(Dropout(0.3))
model_wav2vec.add(Dense(y_train.shape[1], activation='softmax'))  # Output layer

# Compile the model
model_wav2vec.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_wav2vec = model_wav2vec.fit(X_train_wav2vec, y_train, batch_size=64, epochs=50, validation_data=(X_test_wav2vec, y_test))

# Evaluate the model
test_loss_wav2vec, test_accuracy_wav2vec = model_wav2vec.evaluate(X_test_wav2vec, y_test, verbose=1)
print(f"Test Accuracy with Wav2Vec: {test_accuracy_wav2vec * 100:.2f}%")


In [ ]:
# Define the model with extracted VGGish features
model_vggish = Sequential()
model_vggish.add(Dense(256, activation='relu', input_shape=(X_train_vggish.shape[1],)))
model_vggish.add(Dropout(0.3))
model_vggish.add(Dense(128, activation='relu'))
model_vggish.add(Dropout(0.3))
model_vggish.add(Dense(64, activation='relu'))
model_vggish.add(Dropout(0.3))
model_vggish.add(Dense(y_train.shape[1], activation='softmax'))  # Output layer

# Compile the model
model_vggish.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_vggish = model_vggish.fit(X_train_vggish, y_train, batch_size=64, epochs=50, validation_data=(X_test_vggish, y_test))

# Evaluate the model
test_loss_vggish, test_accuracy_vggish = model_vggish.evaluate(X_test_vggish, y_test, verbose=1)
print(f"Test Accuracy with VGGish: {test_accuracy_vggish * 100:.2f}%")


In [ ]:
import torch
from speechbrain.pretrained import EncoderClassifier

# Load the pre-trained emotion recognition model
model = EncoderClassifier.from_hparams(
    source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP",
    savedir="tmp"
)

# Path to the audio file (replace with your file path)
audio_file = "path_to_audio_file.wav"

# Perform emotion recognition
emotion_scores, emotion_index, emotions = model.classify_file(audio_file)

# Display the predicted emotion
print(f"Predicted Emotion: {emotions}")
print(f"Emotion Scores: {emotion_scores}")
